In [4]:
import pandas as pd
import numpy as np
import Clients.OddsApiClient as oddsApi
import sys
from Models.H2hEvent import H2hEvent, H2hBookmaker 
from Transformers.H2hEventTransformer import oddsApiEventsToH2hEvent
import importlib


In [5]:
from datetime import datetime, timedelta, timezone

current_utc_time = datetime.now(timezone.utc)
future_time = current_utc_time + timedelta(hours=48)
iso_formatted_time_with_z = future_time.isoformat(timespec="seconds").replace(
    "+00:00", "Z"
)
current_formatted_time = current_utc_time.isoformat(timespec="seconds").replace(
    "+00:00", "Z"
)
print(iso_formatted_time_with_z)

2025-08-07T02:46:58Z


In [ ]:
importlib.reload(oddsApi)

# sports = oddsApi.ALL_SOCCER_SPORTS + oddsApi.All_OTHER_SPORTS

soccer_events = oddsApi.getEventsForMultipleSports(
    sports=[sport for sport in oddsApi.ALL_SPORTS if "soccer" in sport], regions=[oddsApi.Regions.UK], markets=[oddsApi.Markets.H2H], commenceTimeFrom=current_formatted_time, commenceTimeTo=iso_formatted_time_with_z)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1236139388.py, line 6)

In [13]:
import Betting_Engines.H2hEventAboveMeanOddsBettingEngine as BettingEngine
import sys

from discord_bot import send_message

importlib.reload(BettingEngine)

bettingEngine = BettingEngine.H2hEventAboveMeanOddsBettingEngine(
    alpha=0.01,
    # betOddsUpperLimit=2.5,
    betAmount=10,
    commision=0.0,
    singleBookmaker="betfair_ex_uk"
)

current_utc_time = datetime.now(timezone.utc).strftime("%Y-%m-%d_%H-%M-%S")

result_df = pd.DataFrame(bettingEngine.analyseAndFindBets(oddsApiEventsToH2hEvent(soccer_events)))
csv_url = f'Results/{current_utc_time}_results.csv'
result_df.to_csv(csv_url, index=False) 

# if(len(result_df) > 0):
#     send_message("Results:", csv_url)

print(result_df.to_string())

                                 id      bookmaker       home_team away_team             commence_time  odds  odds_with_commision                    sport_key guessed_outcome  betAmount  kellyBetRatio  halfKellyBetRatio  bookmaker_average_odds  number_of_bookmakers outcome     alpha
0  c8a364012cd5df6bd400a6ca649fe26b  betfair_ex_uk  Inter Miami CF     Pumas 2025-08-06 23:30:00+00:00  2.04                 2.04  soccer_concacaf_leagues_cup    H2hOutcome.H         10       0.037823           0.018911                1.962792                    12    None  0.019282
